In [2]:
import numpy as np
import pandas as pd
import json
import os

In [3]:
folder_path = r"C:\Users\Johnk\OneDrive\Desktop\PhD 2024\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\DE Test\VaryInitFiles\DW"
potential = 'DW'
cutoff = 16

dataframes = {}

folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

converged_runs = []
mean_iters = []
min_e = []
delta_min_e = []
median_e = []
delta_median_e = []
exact_e = []
time_taken = []

num_vqe = []
shots = []
tolerance = []
abs_tolerance = []
strategy = []
popsize = []
maxiter = []


for f in folders:

    data_path = folder_path + "\\{}\\{}_{}.json".format(f,potential, cutoff)
    
    with open(data_path, 'r') as file:
        data = json.load(file)

    num_vqe.append(data['num_VQE'])
    shots.append(data['shots'])

    optimzer = data['Optimizer']
    maxiter.append(optimzer['maxiter'])
    tolerance.append(optimzer['tolerance'])
    abs_tolerance.append(optimzer['abs_tolerance'])
    strategy.append(optimzer['strategy'])
    popsize.append(optimzer['popsize'])

    #success = sum(data['success'])
    
    exact_min_e = np.min(data['exact_eigenvalues'])
    exact_e.append(exact_min_e)

    converged_runs.append(sum(data['success']))
    mean_iters.append(int(np.round(np.mean(data['num_iters']))))
    min_e.append(np.min(data['results']))
    delta_min_e.append(abs(exact_min_e - np.min(data['results'])))
    median_e.append(np.median(data['results']))
    delta_median_e.append(abs(exact_min_e - np.median(data['results'])))
    
    time_taken.append(data['total_run_time'])

    

In [6]:
df = pd.DataFrame({
    'folder': folders,
    'num_vqe': num_vqe,
    'shots': shots,
    'tolerance': tolerance,
    'abs_tolerance': abs_tolerance,
    'strategy': strategy,
    'popsize': popsize,
    'Converged Runs': [f"{c}/40" for c in converged_runs],
    'maxiter': maxiter,
    'Mean Iter': mean_iters,
    #'VQE min E': [f"{num:.5e}" for num in min_e],
    'Delta min E': [f"{num:.5e}" for num in delta_min_e],
    #'VQE median E': [f"{num:.5e}" for num in median_e],
    'Delta median E': [f"{num:.5e}" for num in delta_median_e],
    'Exact': [f"{num:.5e}" for num in exact_e],
    'Time': time_taken
})

In [8]:
df.sort_values(['Delta min E', 'Delta median E'], ascending=[False,False])

,folder,num_vqe,shots,tolerance,abs_tolerance,strategy,popsize,Converged Runs,maxiter,Mean Iter,Delta min E,Delta median E,Exact,Time
0,custom,40,1024,0,0.001,best1bin,15,10/40,5000,4383,7.31637e-14,4.15834e-05,8.91599e-01,0:10:05.007481
1,halton,40,1024,0,0.001,best1bin,15,1/40,5000,4896,7.31637e-14,4.14616e-05,8.91599e-01,2:34:18.205828
2,latinhypercube,40,1024,0,0.001,best1bin,15,3/40,5000,4660,7.31637e-14,4.14616e-05,8.91599e-01,2:33:03.351894
4,sobol,40,1024,0,0.001,best1bin,15,2/40,5000,4775,7.31637e-14,4.14616e-05,8.91599e-01,2:55:04.147325
3,random,40,1024,0,0.001,best1bin,15,0/40,5000,5000,4.13804e-05,4.15022e-05,8.91599e-01,2:38:36.055339
